In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2645)

In [15]:
x[:,:2]

matrix([[ 336858.19604559,  331971.87250321],
        [ 348752.92194979,  341479.79193889],
        [ 626893.78216522,  632977.82093575],
        [ 615162.14549929,  610182.43234577],
        [ 163856.43131575,  160682.72723537],
        [ 233275.03929364,  227018.93831922],
        [ 411153.62524762,  410757.20787492],
        [ 415599.55372876,  415497.60527475],
        [ 383566.46456258,  393736.7804047 ],
        [ 401119.00181937,  395263.57974348],
        [ 426484.60526329,  443310.79505501],
        [ 261929.32097967,  245471.34066777],
        [  92144.33952101,   76289.36065466],
        [ 100005.28371506,  100916.65475016],
        [ 159127.43928515,  167688.43030126],
        [ 151004.68526745,  146887.39218976],
        [ 214425.422214  ,  213802.43466469],
        [ 207388.76818294,  213567.06576677],
        [ 363486.58838623,  380494.34574008],
        [ 352252.20118625,  358629.75027434],
        [ 448553.75912208,  439084.45375577],
        [ 557952.73882521,  555381

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [7e-03, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.92957057e+07  0.00000000e+00  5.50e+03 1.62e+03  1.00e+06     0s
   1  -1.38699854e+07 -3.97525510e+06  2.05e+03 6.06e+02  4.07e+05     0s
   2  -1.02330550e+07 -1.05026314e+07  3.44e+02 1.01e+02  1.19e+05     0s
   3  -9.91857904e+06 -1.03911462e+07  1.64e+01 4.83e+00  1.3

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.6563585712207555e-09,
 186818.22633673414,
 129461.69601975911,
 3.653268413535051e-06,
 70375.21567264557,
 8.523998601659344e-06,
 6.030154551899241e-06,
 2.5447721428580458e-08,
 1.0253639424850611e-10,
 181324.0636889508,
 1.025318709009762e-10,
 1.0253637649139674e-10,
 1.0253513039431489e-10,
 1.0322373175021062e-10,
 92425.99962026141,
 8.750366576688432e-10,
 2.4828851373819765e-07,
 3.7281175156329014e-06,
 89470.49663959067,
 369957.6526398188,
 6.213093167963934e-06,
 110007.38168436792,
 264620.1641929464,
 5.714650847603818e-07,
 59109.22330125904,
 7473.191306260037,
 4057.037245533209,
 190118.04006137064,
 452858.4286183878,
 8.750366946922563e-10,
 5.34527618034961e-07,
 54520.34487055227,
 118904.76625711069,
 1.4112294802350774e-07,
 139614.10016212513,
 3.848530375885769e-06,
 8.748132498915504e-10,
 6.860635991487202e-07,
 6.462984558416179e-07,
 107886.73676417599,
 122149.38511689655,
 3.568238110724341e-05,
 185906.5780507822,
 8.750258323656429e-10,
 4.28958